Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации - выбить auc выше 0.95


In [1]:
!pip install regex
!pip install stop_words
!pip install pymorphy2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

from sklearn import linear_model
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import train_test_split
from collections import defaultdict

import tensorflow as tf
import keras

In [3]:
data = pd.read_excel("отзывы за лето.xls")
data.head(10)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
5,5,Всё удобно норм 👍👍👍,2017-08-14
6,5,Очень удобное приложение.,2017-08-14
7,5,Все устраивает,2017-08-14
8,5,У меня работает все четко. В отличии от банком...,2017-08-14
9,5,Очень все хорошо👍,2017-08-14


In [4]:
exclude = set(punctuation)
sw = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in exclude]
    return " ".join(txt)

data['text'] = data['Content'].apply(preprocess_text)
data = data[data['Rating'] != 3]
data['target'] = (data['Rating'] > 3)*1

In [5]:
data.head(10)

,Rating,Content,Date,text,target
0,5,It just works!,2017-08-14,it just works,1
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,в целое удобноной приложениеиз минус хотеть сл...,1
2,5,Отлично все,2017-08-14,отлично весь,1
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,стать зависать на 1 работа антивирус далёкий н...,1
4,5,"Очень удобно, работает быстро.",2017-08-14,очень удобно работать быстро,1
5,5,Всё удобно норм 👍👍👍,2017-08-14,весь удобно норма 👍👍👍,1
6,5,Очень удобное приложение.,2017-08-14,очень удобный приложение,1
7,5,Все устраивает,2017-08-14,весь устраивать,1
8,5,У меня работает все четко. В отличии от банком...,2017-08-14,у я работать весь чётко в отличие от банкомат ...,1
9,5,Очень все хорошо👍,2017-08-14,очень весь хорошо👍,1


In [6]:
max_words = 3000
max_len = 40
num_classes = 1

# Training
epochs = 5
batch_size = 300
print_batch_n = 100

In [7]:
train, test = train_test_split(data,  random_state=42, test_size=0.2)
train, val  = train_test_split(train, random_state=42, test_size=0.2)

In [8]:
train_corpus = " ".join(train["text"])
# train_corpus = train_corpus.lower()

In [9]:
train_corpus[:100]

'терминал можно сказать всегда с себя очень удобно и просто супер супер очень хороший приложение недо'

In [10]:
from nltk.tokenize import word_tokenize
# nltk.download("punkt")

tokens = word_tokenize(train_corpus)

In [11]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [12]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [13]:
tokens_filtered_top[10:]

['с',
 'что',
 'отлично',
 'спасибо',
 'хороший',
 'нравиться',
 'отличный',
 'это',
 'хорошо',
 'супер',
 'телефон',
 'раз',
 'а',
 'после',
 'по',
 'быть',
 'но',
 'мочь',
 'быстро',
 'за',
 'просто',
 'обновление',
 'при',
 'как',
 'пользоваться',
 'так',
 'не',
 'антивирус',
 'у',
 'только',
 'устраивать',
 'вы',
 'сбербанк',
 'пароль',
 'карта',
 'то',
 'для',
 'свой',
 'нормальный',
 'пока',
 'прошивка',
 'вход',
 'из',
 'без',
 'рута',
 'норма',
 'банк',
 'один',
 'перевод',
 'счёт',
 'разработчик',
 'время',
 'программа',
 'есть',
 'проблема',
 'писать',
 'сделать',
 'деньга',
 'ошибка',
 'бы',
 'довольный',
 'ок',
 'можно',
 'к',
 'он',
 'стать',
 'через',
 'ваш',
 'до',
 'платёж',
 'два',
 'приходиться',
 'вводить',
 'другой',
 'долго',
 'постоянно',
 'или',
 'понятно',
 'уже',
 'мой',
 'ни',
 'надо',
 'код',
 'если',
 'большой',
 'такой',
 'теперь',
 'нужно',
 'последний',
 'исправить',
 'какой',
 'тот',
 'смс',
 'ещё',
 'этот',
 'вылетать',
 'зайти',
 'функция',
 'который',

In [14]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [15]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [16]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in train["text"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in test["text"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in val["text"]], dtype=np.int32)

## Keras model

In [17]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [18]:
num_classes = 2
y_train = keras.utils.to_categorical(train["target"], num_classes)
y_test = keras.utils.to_categorical(test["target"], num_classes)
y_val = keras.utils.to_categorical(val["target"], num_classes)

In [19]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Dropout(0.1))

model.add(Conv1D(128, 3))
model.add(Activation("elu"))
model.add(Dropout(0.3))

model.add(Conv1D(128, 3))
model.add(Activation("elu"))
model.add(Dropout(0.1))

model.add(Conv1D(128, 3))
model.add(Activation("elu"))
model.add(Dropout(0.1))

# model.add(Conv1D(128, 3))
# model.add(Activation("elu"))
# model.add(Dropout(0.1))

# model.add(Conv1D(128, 3))
# model.add(Activation("elu"))
# model.add(Dropout(0.1))

model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

In [20]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['AUC'])

In [21]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1)

Epoch 1/5
43/43 [==============================] - 11s 248ms/step - loss: 0.3737 - auc: 0.9143
Epoch 2/5
43/43 [==============================] - 11s 247ms/step - loss: 0.2208 - auc: 0.9691
Epoch 3/5
43/43 [==============================] - 11s 246ms/step - loss: 0.1662 - auc: 0.9829
Epoch 4/5
43/43 [==============================] - 11s 246ms/step - loss: 0.1296 - auc: 0.9895
Epoch 5/5
43/43 [==============================] - 11s 246ms/step - loss: 0.1081 - auc: 0.9925


In [22]:
score = model.evaluate(x_val, y_val, batch_size=batch_size)
print('\n')
print('Val score:', score[0])
print('Val AUC:', score[1])

11/11 [==============================] - 1s 50ms/step - loss: 0.2042 - auc: 0.9779


Val score: 0.2042361944913864
Val AUC: 0.9778562784194946


2. Предобучаем word2vec и его эмбединга инициализируем сетку, как влияет на качество?

In [23]:
from gensim.models.word2vec import Word2Vec
from gensim.corpora import Dictionary
from multiprocessing import cpu_count

In [24]:
X_train, y_train = train["text"], train["target"]
X_test, y_test = test["text"], test["target"]
X_val, y_val = val["text"], val["target"]

In [25]:
sentences = [[i for i in document.split()] for document in X_train.tolist()]

In [26]:
modelW2V = Word2Vec(sentences=sentences,
                    size=128,
                    window=5,
                    min_count=2,
                    workers=16)

In [27]:
modelW2V.wv.similar_by_word('работа')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('чем', 0.999882161617279),
 ('услуга', 0.9998539090156555),
 ('особенно', 0.9998357892036438),
 ('никакой', 0.9998343586921692),
 ('кто', 0.9998308420181274),
 ('банковский', 0.9998184442520142),
 ('куда', 0.9998089075088501),
 ('жизнь', 0.9997963905334473),
 ('хватать', 0.9997912645339966),
 ('конечно', 0.9997581243515015)]

In [28]:
modelW2V.wv['работа'], modelW2V.wv['работа'].shape


(array([-0.33558774, -0.25091043, -0.03480281,  0.45447245, -0.16399632,
         0.11662161,  0.16853356, -0.18452273,  0.5007739 ,  0.06741212,
        -0.12720552,  0.9026113 ,  0.2992964 , -0.01035859, -0.23664881,
         0.36269456,  0.1174543 , -0.12738502, -0.19900872,  0.42845926,
         0.44716138,  0.133096  , -0.46308753, -0.5358067 , -0.59856606,
         0.6081051 ,  0.3011831 ,  0.38702515, -0.41603938, -0.01834101,
         0.3100412 , -0.03062878, -0.46331662, -0.22630891, -0.08458815,
        -0.33482817,  0.22721097, -0.05469104,  0.04349079, -0.09535503,
        -0.03346722,  0.38855848,  0.2731099 , -0.3465037 , -0.39293402,
        -1.0670491 ,  0.2678034 , -0.04514289,  0.18392427,  0.0836324 ,
        -0.2947632 ,  0.16087766,  0.5274227 , -0.22084978,  0.15511106,
         0.12514882, -0.3127361 ,  0.26786172, -0.76181686,  0.0875733 ,
        -0.00699887, -0.23730157,  0.5052426 ,  0.24675804,  0.20804693,
        -0.18919078, -0.59071654,  0.09452099,  0.2

In [29]:
dictionary = Dictionary(sentences)

In [30]:
WEIGHTS = np.zeros((len(dictionary), 128))
WEIGHTS.shape

(9542, 128)

In [31]:
for token in dictionary:
    if dictionary[token] in modelW2V.wv:
        WEIGHTS[token] = modelW2V.wv[dictionary[token]]

In [32]:
modelW2V = Sequential()
modelW2V.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
modelW2V.add(Dropout(0.1))

modelW2V.add(Conv1D(128, 3))
modelW2V.add(Activation("relu"))
modelW2V.add(Dropout(0.3))

modelW2V.add(Conv1D(128, 1))
modelW2V.add(Activation("relu"))
modelW2V.add(Dropout(0.1))

modelW2V.add(Conv1D(128, 1))
modelW2V.add(Activation("relu"))
modelW2V.add(Dropout(0.1))

modelW2V.add(GlobalMaxPool1D())
modelW2V.add(Dense(10))
modelW2V.add(Activation("relu"))
modelW2V.add(Dropout(0.5))
modelW2V.add(Dense(1))
# modelW2V.add(Dense(num_classes))
modelW2V.add(Activation('sigmoid'))

In [33]:
modelW2V.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['AUC'])

In [34]:
x_train.shape, y_train.shape

((12638, 40), (12638,))

In [35]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history2 = modelW2V.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1)

Epoch 1/5
43/43 [==============================] - 8s 182ms/step - loss: 0.4882 - auc: 0.5406
Epoch 2/5
43/43 [==============================] - 8s 182ms/step - loss: 0.3255 - auc: 0.8462
Epoch 3/5
43/43 [==============================] - 8s 182ms/step - loss: 0.2473 - auc: 0.9223
Epoch 4/5
43/43 [==============================] - 8s 182ms/step - loss: 0.2065 - auc: 0.9453
Epoch 5/5
43/43 [==============================] - 8s 183ms/step - loss: 0.1833 - auc: 0.9580
